In [53]:
from datasets import load_dataset
from vector_db import VectorDB

/Users/jonathansuru/PycharmProjects/OpenLabs/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `hf_hub_download`. Downloading to a local directory does not use symlinks anymore.
  `use_auth_token` will definitely not be supported.
2026-01-10 09:19:47.195 python[2945:173223] 2026-01-10 09:19:47.194341 [W:onnxruntime:, coreml_execution_provider.cc:113 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 20 number of nodes in the graph: 205 number of nodes supported by CoreML: 133
2026-01-10 09:19:47.684 python[2945:173223] 2026-01-10 09:19:47.683995 [W:onnxruntime:, coreml_execution_provider.cc:113 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 112 number of nodes in the graph: 1056 number of nodes supported by CoreML: 739


In [54]:
ds = load_dataset("aimped/medical-translation-test-set")[ "en_fr" ]

# Shuffle the dataset
ds = ds.shuffle(seed=42)
# split into train_set, val_set, test_set
train_size = int(0.8 * len(ds))
val_size = int(0.1 * len(ds))
test_size = len(ds) - train_size - val_size

train_set = ds.select(range(train_size))
val_set = ds.select(range(train_size , train_size + val_size))
test_set = ds.select(range(train_size + val_size , len(ds)))

print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")

Train set size: 839
Validation set size: 104
Test set size: 106


In [55]:
vector_db = VectorDB( )
for item in train_set :
    text = f"English: {item[ 'source' ]} French: {item[ 'target' ]}"
    vector_db.add_entry(text , item[ "target" ])
print("Finished adding entries to the vector database.")

Finished adding entries to the vector database.


In [80]:

import dspy

lm = dspy.LM("openai/zongwei/gemma3-translator:1b" , api_key="ollama" , api_base="http://localhost:11434/v1/")
dspy.configure(lm=lm)

In [81]:

rag = dspy.ChainOfThought("question -> response")

question = "Translate this sentence to French: " + test_set[ "source" ][ 0 ]
rag(question=question)

Prediction(
    reasoning='The data suggests a significantly different peak levels of IL-6, PCT, and CRP between the known pathogen group and the control group. The coagulase-negative group exhibited lower levels of these biomarkers compared to the negative culture group, while the control group had the lowest levels.',
    response='Interleukin (IL-6), procalcitonin (PCT), and C-reactive protein (CRP) levels were highest among the known pathogen group (IL-6 271.8 U/L, PCT 4.6 U/L and CRP 164 mg/L), and were similar between the coagulase-negative Staphylococcus and negative culture groups (IL-6 67.0 U/L versus 61.4 U/L [P=1.00]; PCT 1.0 U/L versus 0.9 U/L [P=0.80]; and CRP 110 mg/L versus 103 mg/L [P=0.75]), and were lowest in the control group (IL-6 31.0 U/L, PCT 0.2 U/L and CRP 41.0 mg/L).'
)

In [58]:
test_set[ "source" ][ 0 ]

'Interleukin (IL)-6, procalcitonin (PCT) and C-reactive protein (CRP) levels were highest among the known pathogen group (IL-6 271.8 U/L, PCT 4.6 U/L and CRP 164 mg/L), were similar between the coagulase-negative Staphylococcus and negative culture groups (IL-6 67.0 U/L versus 61.4 U/L [P=1.00]; PCT 1.0 U/L versus 0.9 U/L [P=0.80]; and CRP 110 mg/L versus 103 mg/L [P=0.75]), and were lowest in the control group (IL-6 31.0 U/L, PCT 0.2 U/L and CRP 41.0 mg/L).'

In [82]:
class RAG(dspy.Module) :
    def __init__(self) :
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self , question) :
        context = vector_db.get_entry(question , top_k=5)
        context = " ".join(context)
        question = "Translate this sentence to French: " + question
        return self.respond(context=context , question=question)

In [83]:
rag = RAG( )
question = test_set[ "source" ][ 0 ]
rag(question=question)

Prediction(
    reasoning='The sentence describes a comparison of serum levels of IL-6, PCT, and CRP across different groups. The levels of these proteins were significantly higher in the “known pathogen group” than in other groups, particularly the coagulase-negative and negative cultures. The levels were relatively similar between the two groups of cultures.  The control group had the lowest levels of all these proteins.',
    response='Le niveau des cytokines IL-6, de la procalcitonine (PCT) et du CRP était le plus élevé parmi les groupes connus de pathogènes (IL-6 271,8 U/L, PCT 4,6 U/L, et CRP 164 mg/L), tandis que les niveaux étaient similaires entre les groupes de culture coagulase-négatifs et négatifs (IL-6 67,0 U/L vs 61,4 U/L [P=1,00]; PCT 1,0 U/L vs 0,9 U/L [P=0,80]; et CRP 110 mg/L vs 103 mg/L [P=0,75]).  Les niveaux étaient les plus bas dans le groupe de contrôle (IL-6 31,0 U/L, PCT 0,2 U/L et CRP 41,0 mg/L).'
)

In [84]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Define an evaluator that we can re-use.


In [88]:
formatted_test_set = [
        dspy.Example(question=item[ 'source' ] , response=item[ 'target' ]).with_inputs('question')
        for item in test_set
]

evaluate = dspy.Evaluate(devset=formatted_test_set , metric=metric , num_threads=1 ,
                         display_progress=True , display_table=2)
# Now run the evaluation with the formatted set
# We reduce num_threads to 1 to see clearer error messages if it fails again
# and use the RAG instance directly.
rag_module = RAG( )
evaluate(rag_module , devset=formatted_test_set , num_threads=1 , metric=metric ,
         display_progress=True , display_table=5)

Average Metric: 10.46 / 20 (52.3%):  18%|█▊        | 19/106 [00:00<00:03, 27.16it/s]

2026/01/10 10:54:39 ERROR dspy.utils.parallelizer: Error for Example({'question': 'A pharmaceutical or veterinary composition as claimed in claim 30 which is adapted for oral administration', 'response': 'Composition pharmaceutique ou vétérinaire selon la revendication 30, qui est adaptée à une administration orale'}) (input_keys={'question'}): Adapter JSONAdapter failed to parse the LM response. 

LM Response: {
  "reasoning": "The system response explicitly states that the composition is adapted for oral administration, directly addressing the core of the question and comparison with the ground truth.",
  "ground_truth_key_ideas": ["administration orale", "pharmaceutical composition", "adapted for oral administration", "formulation designed for oral"]
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
 

Expected to find output fields in the LM response: [reasoning, ground_truth_key_ideas, system_response_key_ideas, discussion, recall, precision] 

Actual output fields parsed from the LM r

Average Metric: 39.54 / 71 (55.7%):  68%|██████▊   | 72/106 [02:52<05:56, 10.47s/it]

2026/01/10 10:57:33 WARNING dspy.utils.parallelizer: SIGINT received. Cancelling.


KeyboardInterrupt: 

In [46]:
import dspy
import sacrebleu

# 1. Configure DSPy with your LLM
# Make sure OPENAI_API_KEY is set in your environment variables
lm = dspy.LM("openai/zongwei/gemma3-translator:1b" , api_key="ollama" , api_base="http://localhost:11434/v1/")
dspy.configure(lm=lm)


# 2. Define the Signature (Input/Output interface)
class FrenchTranslation(dspy.Signature) :
    """Translate English sentences to French accurately and fluently."""
    english_sentence = dspy.InputField(desc="The sentence in English to translate.")
    french_translation = dspy.OutputField(desc="The translation in French.")


# 3. Define the Metric (BLEU Score)
def bleu_metric(gold , pred , trace=None) :
    """
    DSPy metric function.
    - gold: The example from the dataset containing the ground truth.
    - pred: The prediction object from the LLM containing the output.
    """
    # Extract the hypothesis (prediction) and reference (gold)
    hypothesis = [ pred.french_translation ]
    references = [ [ gold.french_translation ] ]

    # Calculate BLEU score using sacrebleu
    bleu = sacrebleu.corpus_bleu(hypothesis , references)
    return bleu.score


# 4. Prepare Data (Trainset)
# DSPy needs a small set of examples to "bootstrap" and optimize the prompt.
trainset = [
        dspy.Example(
                english_sentence="The quick brown fox jumps over the lazy dog." ,
                french_translation="Le vif renard brun saute par-dessus le chien paresseux."
        ).with_inputs("english_sentence") ,

        dspy.Example(
                english_sentence="Hello, how are you?" ,
                french_translation="Bonjour, comment allez-vous ?"
        ).with_inputs("english_sentence") ,

        dspy.Example(
                english_sentence="Machine learning is fascinating." ,
                french_translation="L'apprentissage automatique est fascinant."
        ).with_inputs("english_sentence") ,
]

# 5. Initialize the Student Program
# We start with a basic Predict module.
# DSPy will optimize this by filling in the "few-shot" examples in the prompt.
translator = dspy.Predict(FrenchTranslation)

# 6. Configure the Teleprompter (Optimizer)
# BootstrapFewShot: Selects the best examples from the trainset to include in the prompt
# to maximize the defined metric (BLEU score).
teleprompter = dspy.BootstrapFewShot(
        metric=bleu_metric ,
        max_bootstrapped_demos=3 ,  # Number of examples to include in the final prompt
        max_labeled_demos=1  # Number of fixed examples to always include
)

# 7. Compile (Optimize) the Program
print("Compiling/Optimizing the translator based on BLEU score...")
optimized_translator = teleprompter.compile(student=translator , trainset=trainset)

print("\n--- Optimization Complete ---")

# 8. Test the Optimized Program
test_sentence = "The cat is sleeping on the mat."
print(f"\nSource: {test_sentence}")

# Run the optimized program
result = optimized_translator(english_sentence=test_sentence)

# Print the Result
print(f"\nTranslation: {result.french_translation}")

# --- Safe History Inspection (Debugging) ---
# We wrap this in a check to avoid the TypeError if history is None/empty
history = lm.inspect_history(n=1)
if history :
    print("\n--- Last LLM Call (Prompt) ---")
    # We limit the print length to keep it readable
    print(history[ -1 ][ 'messages' ][ 0 ][ 'content' ][ :500 ] + "...")
else :
    print("\n[Note: LLM history tracking is empty or not available in this configuration]")

# Optional: Calculate BLEU for this specific test if you have a reference
reference = "Le chat dort sur le tapis."
bleu_test = sacrebleu.corpus_bleu([ result.french_translation ] , [ [ reference ] ])
print(f"\nTest BLEU Score (vs reference): {bleu_test.score:.2f}")

Compiling/Optimizing the translator based on BLEU score...


100%|██████████| 3/3 [00:00<00:00, 67.54it/s]

Bootstrapped 2 full traces after 2 examples for up to 1 rounds, amounting to 3 attempts.

--- Optimization Complete ---

Source: The cat is sleeping on the mat.

Translation: Le chat dort sur le tapis.




[2026-01-09T18:21:54.387267]

System message:

Your input fields are:
1. `english_sentence` (str): The sentence in English to translate.
Your output fields are:
1. `french_translation` (str): The translation in French.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## english_sentence ## ]]
{english_sentence}

[[ ## french_translation ## ]]
{french_translation}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Translate English sentences to French accurately and fluently.


User message:

[[ ## english_sentence ## ]]
The quick brown fox jumps over the lazy dog.


Assistant message:

[[ ## french_translation ## ]]
Le rapide renard brun saute par-dessus le chien paresseux.

[[ ## completed ## ]]


Use

In [47]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)

High and low memory usage on Linux are complex and depend heavily on the specific workload and configuration. However, here’s a breakdown of general trends and factors:

**High Memory Usage (Generally indicates potential for performance issues):**

*   **Memory Leaks:** A common problem is memory leaks – where memory is allocated but never released.
*   **Large Applications/Processes:** Applications or processes that consume a lot of memory, especially during long-running tasks, can cause high memory usage.
*   **Memory-Intensive Tasks:** Tasks like video encoding, large database queries, or scientific simulations often have a significant memory footprint.
*   **Unoptimized Code:** Poorly written code can lead to excessive memory allocation.
*   **Insufficient RAM:** If the overall system has insufficient RAM to handle the load, higher memory usage becomes unavoidable.
*   **Data-Driven Systems:**  If a system relies heavily on reading data from disk (e.g., databases), memory pressure 

In [48]:
dspy.inspect_history(n=1)





[2026-01-09T18:27:34.156401]

System message:

Your input fields are:
1. `question` (str):
Your output fields are:
1. `response` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## question ## ]]
{question}

Outputs will be a JSON object with the following fields.

{
  "response": "{response}"
}
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
what are high memory and low memory on linux?

Respond with a JSON object in the following order of fields: `response`.


Response:

{
  "response": "High and low memory usage on Linux are complex and depend heavily on the specific workload and configuration. However, here’s a breakdown of general trends and factors:\n\n**High Memory Usage (Generally indicates potential for performance issues):**\n\n*   **Memory Leaks:** A common proble